In [1]:
import pandas as pd
from sklearn import preprocessing
import numpy
import sys
from statistics import mean

import all_22277_list

import warnings
warnings.filterwarnings('ignore')

In [2]:
all_df = pd.read_csv("./"+"H1N1Data"+".txt",sep='\t',encoding='utf-8')

cols=all_22277_list.cols

#1個list存cols 這邊存的是不同資料時間的tag
print(type(cols))


#把DATA利用cols當作列來排好
all_df=all_df[cols]


<class 'list'>


In [3]:
def PreprocessData(raw_df):

    df=raw_df.drop(['ID'], axis=1)
    ndarray = df.values
    Features = ndarray[:,1:]
    Label = ndarray[:,0]

    minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))
    scaledFeatures=minmax_scale.fit_transform(Features)

    return scaledFeatures,Label

In [4]:
def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return numpy.mean(y_pred == y_true)

In [5]:
#all features是每個人所附帶的資料組
#all label 每筆資料組(人) 的結果1 或 0用來訓練
all_Features,all_Label=PreprocessData(all_df)

'''
前兩筆資料組用來做為測試用
後面全部用來訓練用
'''
test_Features=all_Features[:2]
test_Label=all_Label[:2]
train_Features=all_Features[2:]
train_Label=all_Label[2:]


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

model = Sequential()
model.add(Dense(units=100, input_dim=22277, kernel_initializer='uniform', activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=100, kernel_initializer='uniform', activation='relu'))
model.add(Dense(units=100, kernel_initializer='uniform', activation='relu'))
model.add(Dense(units=100, kernel_initializer='uniform', activation='relu'))
model.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
model.summary()
model.save("DeepFlu4Layer.h5")



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               2227800   
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 2,258,201
Trainable params: 2,258,201
Non-trainable params: 0
______________________________________________

In [7]:
allAcc=[]
allScore=[]

for times in range(100):
    model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
    train_history =model.fit(x=train_Features, y=train_Label, validation_split=0.1, epochs=150, batch_size=200,verbose=0)
    scores=model.evaluate(x=test_Features, y=test_Label)
    all_Features,Label=PreprocessData(all_df)

    #利用網路去預測測試資料
    #predict_classes: 取出屬於第幾類
    all_probability=model.predict_classes(all_Features)
    #predict_proba: 取出每個預測的機率
    all_probability_score=model.predict_proba(all_Features)
    #pd=all_df

    #把預測值存到表格
    #pd.insert(len(all_df.columns), 'probability',all_probability)

    '''
    前兩個是test DATA
    '''
    predict=all_probability[:2]
    predict_score=all_probability_score[:2]
    predict=predict.tolist()
    predict_score=predict_score.tolist()

    testAcc=compute_accuracy(Label[:2],predict[:2])
    allAcc.append(testAcc)
    allScore.append(scores[0])


    #print("iteration:"+str(times)+"\n")
    #print("Label:"+str(int(test_Label[0]))+" "+str(int(test_Label[1]))+"\tpredicted:"+str(predict[0])+"\t"+str(predict[1])+"\tpredicted Prob:"+str(predict_score[0])+"\t"+str(predict_score[1])+"\n")
    #print("testAcc:"+str(testAcc)+"\t Score:"+str(scores)+"\n\n\n")
    if(times==0):
        f1 = open("./"+"deepFlu4LayerResult"+".txt", 'w', encoding = 'UTF-8')
        f1.write("DeepFlu 4 Layer\n========================\n\n\n")
    else:
        f1=open("./"+"deepFlu4LayerResult"+".txt", 'a', encoding = 'UTF-8')
    f1.write("iteration: "+str(times)+"\taccuracy: "+str(testAcc)+"\n")
    #f1.write(str(int(test_Label[0]))+"\t"+str(predict[0])+"\t"+str(predict[1])+"\t"+str(predict_score[0])+"\t"+str(predict_score[1])+"\n")
    f1.close()

1/1 [==============================] - 0s 16ms/step - loss: 0.0060 - accuracy: 1.0000


In [8]:
f1 = open("./"+"deepFlu4LayerResult"+".txt", 'a', encoding = 'UTF-8')
finalAcc=mean(allAcc)
finalScore=mean(allScore)
print("accuracy ALL: "+str(finalAcc)+"\tscore ALL: "+str(finalScore)+"\n")
f1.write("\n========================================\naccuracy ALL: "+str(finalAcc)+"\tscore ALL: "+str(finalScore)+"\n")
    #f1.write(str(int(test_Label[0]))+"\t"+str(predict[0])+"\t"+str(predict[1])+"\t"+str(predict_score[0])+"\t"+str(predict_score[1])+"\n")
f1.close()

print("\nfinish")

accuracy ALL: 0.945	score ALL: 0.2654813621465837


finish
